In [1]:
using CSV, Plots, Random, LinearAlgebra, Statistics, DataFrames, ARFIMA

In [2]:
sentiment = CSV.read("sentiment1.csv")

,TotalSentimentScore,Vaccine,Death,Shut,Date
,Float64,Int64,Int64,Int64,String
1,0.0858615,0,0,0,02/11/2020
2,-0.0980214,2,3,1,02/12/2020
3,-0.0931667,0,3,0,02/13/2020
4,0.00919,0,1,2,02/14/2020
5,0.0,1,0,0,02/15/2020
6,-0.1998,0,0,1,02/16/2020
7,-0.190725,1,0,0,02/17/2020
8,-0.0343133,0,2,0,02/18/2020
9,-0.144262,0,0,0,02/19/2020


In [19]:
stock = CSV.read("Anthem.csv",header = false)

,Column1
,Float64
1,279.91
2,296.0
3,298.87
4,297.82
5,298.978
6,300.135
7,301.293
8,302.45
9,304.06


In [20]:
# index for 5-fold
split = [43,43 + 44,43+ 44*2,43 + 44*3, 43 + 44*4,43 + 44*5]

6-element Array{Int64,1}:
  43
  87
 131
 175
 219
 263

In [24]:
stock[1]

263-element Array{Float64,1}:
 279.910004
 296.0
 298.869995
 297.820007
 298.9775083
 300.1350095
 301.2925108
 302.450012
 304.059998
 293.940002
 294.709991
 288.9733273
 283.2366637
   ⋮
 292.209991
 291.190002
 296.160004
 305.170013
 310.76001
 307.0900067
 303.4200033
 299.75
 298.880005
 285.799988
 273.26001
 272.799988

In [22]:
function abs_err(y,pred)
    return mean(abs.(y-pred))
end

abs_err (generic function with 1 method)

In [41]:
num_of_model = 3
mse = zeros(num_of_model,5)
w = zeros(18)
for i in 1:5
    w_set = []
    # setting train and validation set
    split_cur = split[i]
    split2 = Int(round(split_cur*4/5))
        
    stock_train = stock[1][1:split2]
    stock_val = stock[1][split2 + 1:split_cur]
        
    sent_train = sentiment[1][1:split2]
    sent_val = sentiment[1][split2 + 1:split_cur]
        
    death_train = sentiment[3][1:split2]
    death_val = sentiment[3][split2 + 1:split_cur]
        
    vac_train = sentiment[2][1:split2]
    vac_val = sentiment[2][split2 + 1:split_cur]
        
    shut_train = sentiment[4][1:split2]
    shut_val = sentiment[4][split2 + 1:split_cur]
    
    #Naive Model, w[1:5]
    Xtrain = [sent_train death_train vac_train shut_train ones(length(sent_train))]
    Xval = [sent_val death_val vac_val shut_val ones(length(sent_val))]
    Ytrain = stock_train
    Yval = stock_val
    
    w_cur = Xtrain\Ytrain
    w_set = vcat(w_set, w_cur)
    pred = Xval*w_cur
    mse[1,i] = abs_err(Yval,pred)
    
    #AR1 Model, w[6:11]
    Xtrain = [sent_train[2:end] stock_train[1:end-1] death_train[2:end] vac_train[2:end] shut_train[2:end] ones(length(sent_train)-1)]
    Xval = [sent_val[2:end] stock_val[1:end-1] death_val[2:end] vac_val[2:end] shut_val[2:end] ones(length(sent_val)-1)]
    Ytrain = stock_train[2:end]
    Yval = stock_val[2:end]
    
    w_cur = Xtrain\Ytrain
    w_set = vcat(w_set, w_cur)
    pred = Xval*w_cur
    mse[2,i] = abs_err(Yval,pred)
    
    # AR2 Model, w[12:18]
    Xtrain = [sent_train[3:end] stock_train[2:end-1] stock_train[1:end-2] death_train[3:end] vac_train[3:end] shut_train[3:end] ones(length(sent_train)-2)]
    Xval = [sent_val[3:end] stock_val[2:end-1] stock_val[1:end-2] death_val[3:end] vac_val[3:end] shut_val[3:end] ones(length(sent_val)-2)]
    Ytrain = stock_train[3:end]
    Yval = stock_val[3:end]
    
    w_cur = Xtrain\Ytrain
    w_set = vcat(w_set, w_cur)
    pred = Xval*w_cur
    mse[3,i] = abs_err(Yval,pred)
    w = hcat(w,w_set)
end

[206.960007, 200.869995, 191.589996, 185.9533283, 180.3166607, 174.679993, 196.839996]
[237.2182624266238, 224.26490817539917, 217.5629274955046, 205.2170738842855, 185.6993023891064, 174.42066333846378, 175.44544300219096]
[261.549988, 264.119995, 266.099996, 268.079997, 270.059998, 268.220001, 280.609985, 280.730011, 269.570007, 270.0466713, 270.5233357, 271.0, 275.070007, 270.619995, 265.730011]
[254.30948519379075, 250.67284096918615, 267.3817297963545, 265.1383951792109, 263.8995264368742, 268.34368545084186, 267.6484190466771, 272.6912557651994, 274.10533000154106, 266.41123890644457, 270.99772405585026, 262.09046904132475, 270.03298575094243, 275.60218938877955, 268.785887405016]
[291.369995, 294.109985, 292.7499897, 291.3899943, 290.029999, 294.200012, 289.269989, 282.089996, 287.890015, 292.123342, 296.356669, 300.589996, 291.01001, 288.540009, 264.540009, 265.73999, 264.6033223, 263.4666547, 262.329987, 267.429993, 265.720001, 265.529999, 266.429993, 266.0233257]
[285.8558044

In [42]:
mse

3×5 Array{Float64,2}:
 51.9352  28.0873   13.6328   9.45674  9.12706
 18.501    6.01871   5.02105  3.52973  3.36717
 17.9895   5.52168   4.81623  3.65989  3.3327

In [43]:
avg_mse = mean(mse, dims=2)

3×1 Array{Float64,2}:
 22.44780794130933
  7.287525197851982
  7.0639976249576435

In [49]:
final_model = findmin(avg_mse)[2][1]
test_mse = zeros(5)
model = ["model 1", "model 2", "model 3"]
for i in 1:5
    #Fixed index
    stock_test = stock[1][split[i]+1:split[i+1]]
    sent_test = sentiment[1][split[i]+1:split[i+1]]
    death_test = sentiment[3][split[i]+1:split[i+1]]
    vac_test = sentiment[2][split[i]+1:split[i+1]]
    shut_test = sentiment[4][split[i]+1:split[i+1]]
    
    if final_model == 1
        Xtest = [sent_test death_test vac_test shut_test ones(length(sent_test))]
        Ytest = stock_test
    
        w_cur = w[1:5, i+1]
        pred = Xtest*w_cur
        test_mse[i] = abs_err(Ytest,pred)
        
    elseif final_model == 2
        Xtest = [sent_test[2:end] stock_test[1:end-1] death_test[2:end] vac_test[2:end] shut_test[2:end] ones(length(stock_test)-1)]
        Ytest = stock_test[2:end]
        
        w_cur = w[6:11, i+1]
        pred = Xtest*w_cur
        test_mse[i] = abs_err(Ytest,pred)
        
    elseif final_model == 3
        Xtest = [sent_test[3:end] stock_test[2:end-1] stock_test[1:end-2] death_test[3:end] vac_test[3:end] shut_test[3:end] ones(length(stock_test)-2)]
        Ytest = stock_test[3:end]
        
        w_cur = w[12:18, i+1]
        pred = Xtest*w_cur
        test_mse[i] = abs_err(Ytest,pred)
    else
        print("error")
    end
end
print("Selected: ", model[final_model], ", estimated error is: ",mean(test_mse), ", Validation Error is :", findmin(avg_mse)[1])

Selected: model 3, estimated error is: 6.0059497268731, Validation Error is :7.0639976249576435